# QA - Quantum Annealing - Dwave 101

Documento referencial fundamental:

https://support.dwavesys.com/hc/en-us/articles/360045654674-Which-Solver-Sampler-Should-I-Use-

Documentos iniciales esencial sobre solvers, tipos etc:

https://docs.dwavesys.com/docs/latest/doc_getting_started.html


https://docs.dwavesys.com/docs/latest/c_solver_parameters.html#:~:text=1000%2C%20**reverse_anneal_params)-,annealing_time,returned%20in%20the%20timing%20structure.


#### Conceptos previos

El annealing cuántico en el que se fundamentan los QPU de Dwave, está basado en la computación adiabática, con un hamiltoniano inicial de fácil preparación, y al que se hace evolucionar en el tiempo hacía el hamiltoniano problema, siendo la función de coste a minimizar, la energía de este hamiltoniano problema. En un annealer el principio de evolución adiabática solo es inspiradora, por lo que no está garantizada que el estado final corresponda al de mínima energía del problema planteado. 

En el siguiente enlace se desarrolla un poco más esta base fundacional.

https://docs.dwavesys.com/docs/latest/c_qpu_annealing.html

En D-Wave, un `solver` es una abstracción de alto nivel que abarca los componentes hardware y software necesarios para resolver un problema de optimización determinado. Consiste en una colección de herramientas y algoritmos que se utilizan para mapear el problema de optimización en el hardware del annealer.

El solver encapsula todo el proceso de formulación de la función de coste del problema,y de sus restricciones, brindando un interfaz que permita a los usuarios ingresar el problema de una manera que se pueda luego transpilar en el annealer cuántico.  

Los solvers también manejan varios detalles técnicos, como configurar la conexión con el annealer, envío a la nube para su ejecución y recuperación de los resultados.

Documentación específica:

Getting Started oficial:

https://docs.dwavesys.com/docs/latest/doc_getting_started.html

Parámetros Solvers:

https://docs.dwavesys.com/docs/latest/doc_solver_ref.html


Por otro lado, un `sampler` es un componente de bajo nivel que interactúa directamente con el hardware del annealer. Toma un problema como entrada y devuelve una muestras de la distribución de posibles soluciones. Un sampler no realiza ninguna formulación o interpretación del problema de alto nivel, ni aplica ninguna restricción a las soluciones que devuelve. Simplemente toma un problema en un formato particular, lo mapea en el annealer cuántico, lo ejecuta n `reads`, equivalente a los `runs`, y devuelve el conjunto de muestras o `samples`, que son la configuraciones de los qbits-variables soluciones del problema.


La adaptación del problema al hardware qpu, o transpilación, se denomina `embedding` o incrustación en este ecosistema.


Algoritmos de grafos proporcionados

https://docs.ocean.dwavesys.com/projects/dwave-networkx/en/latest/reference/algorithms/index.html


Instalación del entorno SDK. Mejor crear env Anaconda.

In [1]:
# pip install dwave-ocean-sdk==6.0.1

Configuración de la cuenta. 
Mejor hacerlo desde consola para mayor claridad. Activar previamente env ocean!

Solo es necesario la primera vez

In [2]:
# !dwave config create -> mejor desde consola

Dentro del SDK Ocean, la biblioteca `dimod` es una API para samplers, proporcionado por ejemplo, la clase BQM (binary quadratic model) que permite plantear problemas de optimización usando:

- Modelo Ising para variables {-1,1}
- Modelos Qubo, tanto cuadráticos como de orden superior!- para variables {0,1}

https://docs.ocean.dwavesys.com/en/stable/docs_dimod/index.html


Ocean proporciona varios samplers que permiten depurar los algoritmos con cpu local, y que no consumen por tanto tiempo-máquina:

Una lista de los mismos:

https://docs.ocean.dwavesys.com/en/stable/docs_dimod/reference/sampler_composites/samplers.html

Pero especialmente, estamos interesados en samplers Qhardware:

https://docs.ocean.dwavesys.com/en/stable/docs_system/reference/samplers.html

### Primer ejemplo. Maxcut 

Se pretende resolver un problema maxcut de 5 nodos. Para fijar visualmente el problema, una geometría de prisma de base cuadrada:  4 nodos en los vértices de las aristas de un cuadrado, y un quinto nodo en el centro. 

#### Modelo Ising

El formalismo Ising se adapta especialmente bien a este tipo de problemas, favoreciendo con un peso -1 a las configuraciones nodales de diferente clase, o penalizando con un peso de +1 cuando los nodos pertenecen a la misma clase.

Por cada configuración se obtendrá una función de coste asociada, con el balance final entre premios y penalizanciones para las diferentes aristas de la geometría nodal.

Se utiliza QPU real Dwave con el sampler DwaveSampler

Los parámetros del problema: J, h, BQM etc se pueden consultar en el siguiente link:

https://docs.dwavesys.com/docs/latest/c_solver_problems.html

In [3]:
# Modelo Maxcut Ising 

# 4 nodos en los vértices de un cuadrado, con nodo central, o prisma de base cuadrada.

import dimod

J = {(0,1):1, (0,3):1, (0,4):1,(1,2):1, (1,4):1, (2,3):1, (2,4):1, (3,4):1}
h = {}

problem = dimod.BinaryQuadraticModel(h, J, 0.0, dimod.SPIN)  #BQM
print("El problema a resolver es::")
print(problem)


El problema a resolver es::
BinaryQuadraticModel({0: 0.0, 1: 0.0, 3: 0.0, 4: 0.0, 2: 0.0}, {(1, 0): 1.0, (3, 0): 1.0, (4, 0): 1.0, (4, 1): 1.0, (4, 3): 1.0, (2, 1): 1.0, (2, 3): 1.0, (2, 4): 1.0}, 0.0, 'SPIN')


In [ ]:
import dimod

G = nx.Graph()
G.add_weighted_edges_from({(0, 1, .1), (0, 2, .5), (0, 3, .1), (1, 2, .1),(1, 3, .5), (2, 3, .1)})

dnx.traveling_salesperson(G, dimod.ExactSolver(), start=0) 
[0, 1, 2, 3]

In [4]:
from dwave.system import DWaveSampler
from dwave.system import EmbeddingComposite


sampler = EmbeddingComposite(DWaveSampler())

result = sampler.sample(problem, num_reads=10)

print("Soluciones Ising")
print(result)

Soluciones Ising
   0  1  2  3  4 energy num_oc. chain_.
0 -1 +1 -1 +1 -1   -4.0       4     0.0
1 +1 -1 +1 -1 +1   -4.0       2     0.0
2 +1 -1 +1 -1 -1   -4.0       3     0.0
3 -1 +1 -1 +1 +1   -4.0       1     0.0
['SPIN', 4 rows, 10 samples, 5 variables]


Cada arista maxcut contribuye con -1 a la energía, cada arista no maxcut contribuye con +1.

Energía mínima óptima = aristas * -1 = -8

Energía mínima alcanzada = -4

dif: 8-4 = 4

max cut =  dif + dif/2 = 6 aristas

max cut es de 6, con 4 posibles combinaciones nodales óptimas. 

Veamos ahora un modelo BQM con restricciones:

- Modelo CQM, cuadrático con restricciones

#### Modelo BQM

Veamos ahora un modelo BQM con restricciones, o modelo CQM (modelo cuadrático con restricciones

In [55]:
# Modelo binario cuadrático con restricciones: CQM

# Definición de las variables binarias {0,1}

x0 = dimod.Binary("x0")
x1 = dimod.Binary("x1")
x2 = dimod.Binary("x2")
x3 = dimod.Binary("x3")

In [56]:
blp = dimod.ConstrainedQuadraticModel() #Modelo CQM

blp.set_objective(-2*x0-3*x1+4*x2-x3)
blp.add_constraint(x0 + x1 <= 2, "Primera restricción")
blp.add_constraint(x2 + x3 <= 1, "Segunda restricción")

'Segunda restricción'

In [57]:
print("Variables:")
print(blp.variables)
print("Objetivo:")
print(blp.objective)
print("Restricciones:") 
print(blp.constraints)

Variables:
Variables(['x0', 'x1', 'x2', 'x3'])
Objetivo:
ObjectiveView({'x0': -2.0, 'x1': -3.0, 'x2': 4.0, 'x3': -1.0}, {}, 0.0, {'x0': 'BINARY', 'x1': 'BINARY', 'x2': 'BINARY', 'x3': 'BINARY'})
Restricciones:
{'Primera restricción': Le(ConstraintView({'x0': 1.0, 'x1': 1.0}, {}, 0.0, {'x0': 'BINARY', 'x1': 'BINARY'}), 2.0), 'Segunda restricción': Le(ConstraintView({'x2': 1.0, 'x3': 1.0}, {}, 0.0, {'x2': 'BINARY', 'x3': 'BINARY'}), 1.0)}


- Ensayos de soluciones (samples)

In [60]:
sample1 = {"x0":1, "x1":1, "x2":1, "x3":1}
print("Muestra", sample1)
print("Su coste", blp.objective.energy(sample1))
print("Es factible?",blp.check_feasible(sample1))
print("Métrica restricciones:")
print(blp.violations(sample1))

Muestra {'x0': 1, 'x1': 1, 'x2': 1, 'x3': 1}
Su coste -2.0
Es factible? False
Métrica restricciones:
{'Primera restricción': 0.0, 'Segunda restricción': 1.0}


In [61]:
sample2 = {"x0":0, "x1":0, "x2":1, "x3":1}
print("Muestra", sample2)
print("Su coste", blp.objective.energy(sample2))
print("Es factible?",blp.check_feasible(sample2))
print("Métrica restricciones:")
print(blp.violations(sample2))

Muestra {'x0': 0, 'x1': 0, 'x2': 1, 'x3': 1}
Su coste 3.0
Es factible? False
Métrica restricciones:
{'Primera restricción': -2.0, 'Segunda restricción': 1.0}


En los resultados anteriores:
- métrica con valor negativo: se viola restricción por defecto 
- métrica con valor positiva: se viola restricción por exceso


A continuación se usará un Solver CQM exacto, por fuerza bruta. 

Lo que hace este solver es barrer todo el espacio de entradas, y para cada uno, valorar la función de coste, es decir, su función objetivo

In [62]:
# Solver fuerza bruta ExactCQMSolver

solver = dimod.ExactCQMSolver()
solution = solver.sample_cqm(blp)

print("Lista de asignaciones")
print(solution)

The list of assignments is
   x0 x1 x2 x3 energy num_oc. is_sat. is_fea.
11  1  1  0  1   -6.0       1 arra...    True
3   1  1  0  0   -5.0       1 arra...    True
9   0  1  0  1   -4.0       1 arra...    True
1   0  1  0  0   -3.0       1 arra...    True
10  1  0  0  1   -3.0       1 arra...    True
2   1  0  0  0   -2.0       1 arra...    True
15  1  1  1  1   -2.0       1 arra...   False
7   1  1  1  0   -1.0       1 arra...    True
8   0  0  0  1   -1.0       1 arra...    True
0   0  0  0  0    0.0       1 arra...    True
13  0  1  1  1    0.0       1 arra...   False
5   0  1  1  0    1.0       1 arra...    True
14  1  0  1  1    1.0       1 arra...   False
6   1  0  1  0    2.0       1 arra...    True
12  0  0  1  1    3.0       1 arra...   False
4   0  0  1  0    4.0       1 arra...    True
['INTEGER', 16 rows, 16 samples, 4 variables]



De la tabla anterior se observa que la solución óptima, función de coste mínima,  corresponde a:

[x0x1x2x3x4]=[1101]

con valor f=-6.

Se pueden extraer las soluciones factibles con el siguiente método:

In [64]:
feasible_sols = solution.filter(lambda s: s.is_feasible)
feasible_sols.first

Sample(sample={'x0': 1, 'x1': 1, 'x2': 0, 'x3': 1}, energy=-6.0, num_occurrences=1, is_satisfied=array([ True,  True]), is_feasible=True)

#### CQM a Qubo

Plantear un problema en formalismo qubo obliga a incorporar las restricciones a la función objetivo como penalizaciones,  usando para ello expresiones cuadráticas, generalmente con la ayuda de `variables slack`, y un `multiplicador o factor de Lagrange` que module el nivel de penalización por incumplimiento de la restricción.


In [67]:
# vy0, y1 = dimod.Binaries(["y0", "y1"])
y0 = dimod.Binary("y0")
y1 = dimod.Binary("y1")

cqm = dimod.ConstrainedQuadraticModel() #CQM

cqm.set_objective(-2*y0-3*y1)
cqm.add_constraint(y0 + 2*y1 <= 2);

In [68]:
qubo, invert = dimod.cqm_to_bqm(cqm, lagrange_multiplier = 5)
print(qubo)

BinaryQuadraticModel({'y0': -17.0, 'y1': -23.0, 'slack_v78a5140c4d30443b8e6c6f67b113bb37_0': -15.0, 'slack_v78a5140c4d30443b8e6c6f67b113bb37_1': -15.0}, {('y1', 'y0'): 20.0, ('slack_v78a5140c4d30443b8e6c6f67b113bb37_0', 'y0'): 10.0, ('slack_v78a5140c4d30443b8e6c6f67b113bb37_0', 'y1'): 20.0, ('slack_v78a5140c4d30443b8e6c6f67b113bb37_1', 'y0'): 10.0, ('slack_v78a5140c4d30443b8e6c6f67b113bb37_1', 'y1'): 20.0, ('slack_v78a5140c4d30443b8e6c6f67b113bb37_1', 'slack_v78a5140c4d30443b8e6c6f67b113bb37_0'): 10.0}, 20.0, 'BINARY')


In [69]:
# Transpilado  (embedding) a Qpu

sampler = EmbeddingComposite(DWaveSampler())
result = sampler.sample(qubo, num_reads=10)

print("Soluciones encontradas:")
print(result)

The solutions that we have obtained are
  slack_v78a5140c4d30443b8e6c6f67b113bb37_0 ... y1 energy num_oc. chain_.
0                                         0 ...  1   -3.0       6     0.0
1                                         0 ...  0   -2.0       1     0.0
2                                         1 ...  0   -2.0       2     0.0
3                                         0 ...  1    0.0       1     0.0
['BINARY', 4 rows, 10 samples, 4 variables]


Pero no todas estas soluciones son factibles, cumplen las restricciones, aunque su función de coste pueda ser la mínima.

A continuación se hace un filtrado de estos resultados:

In [78]:
samples = []
occurrences = []
for s in result.data():
    samples.append(invert(s.sample))
    occurrences.append(s.num_occurrences)
sampleset = dimod.SampleSet.from_samples_cqm(samples,cqm,
    num_occurrences=occurrences)
print("Soluciones (función de coste vs factibilidadd):")
print(sampleset)

Soluciones (función de coste vs factibilidadd):
  y0 y1 energy num_oc. is_sat. is_fea.
3  1  1   -5.0       1 arra...   False
0  0  1   -3.0       6 arra...    True
1  1  0   -2.0       1 arra...    True
2  1  0   -2.0       2 arra...    True
['INTEGER', 4 rows, 10 samples, 2 variables]


In [86]:
final_sols = sampleset.filter(lambda s: s.is_feasible)
final_sols = final_sols.aggregate()
print("\nSoluciones finales:")
print(final_sols)


Soluciones finales:
  y0 y1 energy num_oc. is_sat. is_fea.
0  0  1   -3.0       6 arra...    True
1  1  0   -2.0       3 arra...    True
['INTEGER', 2 rows, 9 samples, 2 variables]



### Caracterización física de QPUs asignables a Cliente

La instrucción `!dwave config create` , que ha de ejecutarse una primera y única vez en `local`, o en cada sesión si se hace desde `colab`, instancia el objeto Client que configura el entorno operativo y la disponibilidad de recursos asociados.

Se puede obtener info relevante usando diferentes métodos del mismo.

In [21]:
# Funciones auxiliares informativas

import random
from dwave.system import DWaveSampler
from dwave.cloud import Client

def client_info():
    print("Solvers:")
    for solver in Client.from_config().get_solvers():
        print(solver)

def dwave_info(sampler,modo=0):
    print("Nombre:",sampler.properties["chip_id"])
    print("No. qubits:",sampler.properties["num_qubits"])
    print("Categoría:",sampler.properties["category"])
    print("Problemas soportados:",sampler.properties["supported_problem_types"])
    print("Topología:",sampler.properties["topology"])
    print("Fuerza de acoplamiento", sampler.properties["h_range"])
    print("Rango de 'reads':",sampler.properties["num_reads_range"])
    print("Annealing time (defecto)",sampler.properties["default_annealing_time"],"microsecs")
    print("Rango annealing time (us)",sampler.properties["annealing_time_range"])
    if modo:
        print("Acoplamientos:",sampler.properties["couplers"]) #muestra geometría
        print(sampler.adjacency) # muestra adyacencias

In [22]:
with Client.from_config() as client:
    solver = client.get_solver()
    # Build a random Ising model to exactly fit the graph the solver supports
    linear = {index: random.choice([-1, 1]) for index in solver.nodes}
    quad = {key: random.choice([-1, 1]) for key in solver.undirected_edges}

In [25]:
quad

{(827, 830): -1,
 (247, 255): -1,
 (266, 268): -1,
 (1282, 1410): 1,
 (1920, 1926): 1,
 (1639, 1647): 1,
 (1979, 1981): -1,
 (262, 270): -1,
 (561, 566): 1,
 (602, 604): -1,
 (1347, 1475): 1,
 (321, 325): -1,
 (1975, 1983): -1,
 (1994, 1996): -1,
 (1713, 1717): 1,
 (897, 902): -1,
 (1683, 1811): -1,
 (336, 340): -1,
 (616, 623): 1,
 (908, 916): 1,
 (1768, 1774): -1,
 (672, 676): 1,
 (952, 959): -1,
 (971, 972): -1,
 (1251, 1255): 1,
 (616, 744): -1,
 (690, 693): -1,
 (110, 118): -1,
 (1145, 1273): -1,
 (1244, 1252): -1,
 (1307, 1308): -1,
 (1587, 1591): 1,
 (952, 1080): -1,
 (1026, 1029): 1,
 (446, 454): -1,
 (745, 750): -1,
 (1309, 1317): -1,
 (154, 282): 1,
 (683, 811): -1,
 (1321, 1327): -1,
 (1362, 1365): -1,
 (1546, 1674): 1,
 (1081, 1086): -1,
 (243, 246): 1,
 (3, 5): -1,
 (866, 868): -1,
 (1019, 1147): -1,
 (1657, 1663): -1,
 (1096, 1101): -1,
 (1882, 2010): -1,
 (856, 860): -1,
 (298, 303): -1,
 (58, 62): -1,
 (1731, 1733): 1,
 (1450, 1454): -1,
 (1491, 1492): 1,
 (634, 639): 1

In [18]:
#solver.undirected_edges

In [125]:
# Obtenemos solvers disponibles
client_info()

Solvers:
StructuredSolver(id='DW_2000Q_6')
BQMSolver(id='hybrid_binary_quadratic_model_version2')
DQMSolver(id='hybrid_discrete_quadratic_model_version1')
CQMSolver(id='hybrid_constrained_quadratic_model_version1')
StructuredSolver(id='Advantage_system6.1')
StructuredSolver(id='Advantage2_prototype1.1')
StructuredSolver(id='Advantage_system4.1')


Los solvers anteriores tienen asociado samplers cuyas características se pueden consultar:

In [122]:
sampler=DWaveSampler(solver='DW_2000Q_6')
dwave_info(sampler)

Nombre: DW_2000Q_6
No. qubits: 2048
Categoría: qpu
Problemas soportados: ['ising', 'qubo']
Topología: {'type': 'chimera', 'shape': [16, 16, 4]}
Fuerza de acoplamiento [-2.0, 2.0]
Rango de 'reads': [1, 10000]
Annealing time (defecto) 20.0 microsecs
Rango annealing time (us) [1.0, 2000.0]


In [123]:
sampler=DWaveSampler(solver='Advantage_system4.1')
dwave_info(sampler)

Nombre: Advantage_system4.1
No. qubits: 5760
Categoría: qpu
Problemas soportados: ['ising', 'qubo']
Topología: {'type': 'pegasus', 'shape': [16]}
Fuerza de acoplamiento [-4.0, 4.0]
Rango de 'reads': [1, 10000]
Annealing time (defecto) 20.0 microsecs
Rango annealing time (us) [0.5, 2000.0]


In [124]:
sampler=DWaveSampler(solver='Advantage2_prototype1.1')
dwave_info(sampler)

Nombre: Advantage2_prototype1.1
No. qubits: 576
Categoría: qpu
Problemas soportados: ['ising', 'qubo']
Topología: {'type': 'zephyr', 'shape': [4, 4]}
Fuerza de acoplamiento [-4.0, 4.0]
Rango de 'reads': [1, 10000]
Annealing time (defecto) 20.0 microsecs
Rango annealing time (us) [1.0, 2000.0]


In [126]:
sampler=DWaveSampler(solver='Advantage_system6.1')
dwave_info(sampler)

Nombre: Advantage_system6.1
No. qubits: 5760
Categoría: qpu
Problemas soportados: ['ising', 'qubo']
Topología: {'type': 'pegasus', 'shape': [16]}
Fuerza de acoplamiento [-4.0, 4.0]
Rango de 'reads': [1, 10000]
Annealing time (defecto) 20.0 microsecs
Rango annealing time (us) [0.5, 2000.0]


### Maxcut con QPU

Retomando el problema maxcut de comienzo, se va a resolver ahora con qpu.-

In [4]:
# Problema maxcut de 5 nodos 

J = {(0,1):1, (0,3):1, (0,4):1,(1,2):1, (1,4):1, (2,3):1, (2,4):1, (3,4):1}
h = {}

prisma = dimod.BinaryQuadraticModel(h, J, 0.0, dimod.SPIN)

# embedding y run en annealer DW_2000Q_6

sampler = EmbeddingComposite(DWaveSampler(solver = "DW_2000Q_6"))
result = sampler.sample(prisma, num_reads=10, 
    return_embedding = True)


In [5]:
print("'Samples' obtenidos:\n")
print(result)
print("\nLa incrustación usada fue:\n")
print(result.info["embedding_context"])

'Samples' obtenidos:

   0  1  2  3  4 energy num_oc. chain_.
0 +1 -1 +1 -1 +1   -4.0       2     0.0
1 -1 +1 -1 +1 -1   -4.0       2     0.0
2 -1 +1 -1 +1 +1   -4.0       1     0.0
3 +1 -1 +1 -1 -1   -4.0       5     0.0
['SPIN', 4 rows, 10 samples, 5 variables]

La incrustación usada fue:

{'embedding': {1: (238,), 0: (232,), 3: (236,), 4: (237, 235), 2: (233,)}, 'chain_break_method': 'majority_vote', 'embedding_parameters': {}, 'chain_strength': 2.529440096147762}


### Annealing time

Como se mencionó al comienzo del cuaderno, el annealer hace evolucionar un hamiltoniano inicial, de setup, a uno final, que representa el problema a minimizar.

Esta evolución temporal queda caracterizada por el  `annealing time`, uno de los parámetros fundamentales de un determinado `read`.

El rango de valores admitidos depende del qpu seleccionado. En el  DW_2000Q_6  por ejemplo varía entre 1 y 2000 𝜇𝑠, con una resolución de 0,02 𝜇𝑠, siendo 20 𝜇𝑠 el valor por defecto. Consultar la descripción proporcionada por `dwave_info(sampler)` para otros samplers, y el siguiente doc:

https://docs.dwavesys.com/docs/latest/c_solver_parameters.html#annealing-time

Se va a repetir el ejemplo anterior usando un At= 100𝜇𝑠 


In [114]:
# DW_2000Q_6 con annealing_time =100 us

J = {(0,1):1, (0,3):1, (0,4):1,(1,2):1, (1,4):1, (2,3):1, (2,4):1, (3,4):1}
h = {}

prisma = dimod.BinaryQuadraticModel(h, J, 0.0, dimod.SPIN)

sampler = EmbeddingComposite(DWaveSampler(solver = "DW_2000Q_6"))
result = sampler.sample(prisma, num_reads=10, 
    return_embedding = True)
result = sampler.sample(prisma, num_reads=10, annealing_time = 100)

print("'Samples' obtenidos:\n")
print(result)

'Samples' obtenidos:

   0  1  2  3  4 energy num_oc. chain_.
0 +1 -1 +1 -1 -1   -4.0       5     0.0
1 -1 +1 -1 +1 -1   -4.0       1     0.0
2 +1 -1 +1 -1 +1   -4.0       4     0.0
['SPIN', 3 rows, 10 samples, 5 variables]


#### Programación hacía adelante (`forward sheduling`)

La evolución temporal  del hamiltoniano durante el `annealing time` es lineal, con una pendiente constante entre t=0 y el tiempo de annealing. Durante este intervalo el parámetro del hamiltoniano, s(t),  varía linealmente entre s=0 y s=1, de modo que:
                
            H(s) = sH0 + (1-s)Hf, s(t) = (1/ta)* t, ta = annealing time

Pero este mapeo lineal se puede cambiar fijando los valores de s para determinado tiempo de annealing.

Se usa para ello listas con pares de números en coma flotante. El primer elemento del par es el tiempo 𝑡 en microsegundos con una granularidad de 0.01 u 0.0.2 𝜇𝑠 según sampler, y el segundo elemento, su valor `s` en ese instante. 

La pendiente máxima de cualquier segmento de curva no debe ser mayor que el inverso del tiempo mínimo soportado por el annealer. 
Así, para una QPU con rango de annealing de 0.5-2000 𝜇𝑠, la pendiente máxima que nunca se debe superar es:

- m= (sf-si)/(tf-ti)=(1-0)/(0.5-0)=2𝜇𝑠−1

Así, entre los dos intervalos siguientes: [0.0, 0.0], [5.0, 0.25], el annealer empieza en t=0 con s=0, variando este parámetro linealmente hasta el valor s=0.25 para t=5 us. La pendiente de intervalo es por tanto: 

- m = (0.25-0/5-0)= 0.05 𝜇𝑠−1

El mapeo s(t) resultante es la curva lineal por partes que interconecta los puntos proporcionados, y que determinan la rapidez con la que evoluciona el annealer en cada intervalo

En el siguiente doc se puede ampliar la info sobre este proceso:

https://docs.dwavesys.com/docs/latest/c_solver_parameters.html#param-anneal-sched

#### Importante: 

Recordemos que por el principio adiabático, una evolución demasiado rápida puede provocar que el estado final no corresponda al de mínima energía

A continuación se repetirá el problema maxcut anterior, usando una programación de 4 etapas, con una pronunciada pendiente inicial, muy suave entre 10-40𝜇𝑠, y de nuevo pronunciada en los últimos 10𝜇s, alcanzándose el hamiltoniano final a los 50 𝜇𝑠 (s=1) 

In [118]:
forward_schedule=[[0.0, 0.0], [10.0, 0.25], [40, 0.75], [50, 1.0]]

sampler = EmbeddingComposite(DWaveSampler())

result = sampler.sample(prisma, num_reads=10, 
    anneal_schedule = forward_schedule)

print("'Samples' obtenidos:\n")
print(result)

'Samples' obtenidos:

   0  1  2  3  4 energy num_oc. chain_.
0 +1 -1 +1 -1 +1   -4.0       2     0.0
1 +1 -1 +1 -1 -1   -4.0       3     0.0
2 -1 +1 -1 +1 -1   -4.0       3     0.0
3 -1 +1 -1 +1 +1   -4.0       2     0.0
['SPIN', 4 rows, 10 samples, 5 variables]


### Programación inversa (`reverse sheduling`)

El annealer también soporta programación inversa, es decir, tramos en los cuales la variación s(t) es de la forma:
                
                s(t) = 1- 1/ta *t

No obstante, una programación de este tipo debe de empezar y terminar con s=1, por lo que es imperativo que el último tramo sea de programación directa.

Este modo también obliga a indicar un estado inicial para s=1. Se hará mediante parejas clave:valor (índice_qb, estado) 

- -1 / 1 : Ising, activos
- 0 / 1 : QUBO, activos
- 3      : sin usar o inactivos

https://docs.dwavesys.com/docs/latest/c_solver_parameters.html#param-initial-state

Si se han programado múltiples `reads` mediante una llamada única a la API del solver, existen dos enfoques para el estado inicial del siguiente `read`:

- `reinitialize_state=true`: reinicializa al estado inicial especificado en cada `read`.

- `reinitialize_state=false`: solo se fija el estado inicial en el primer read. Los siguientes parten `del estado final` del `read` anterior.

La programación siguiente parte en t=0 con s=1,  haciendo una programación inversa de 10𝜇𝑠 hasta s=0.5,  y a continuación una directa para completar el annealing a los 20𝜇𝑠, regresando a  s=1.


In [121]:
reverse_schedule=[[0.0, 1.0], [10.0, 0.5], [20, 1.0]]
estado_inicial = {0:-1, 1:-1, 2:-1, 3:1, 4:1}  # q4q3q2q1q0=-1-1-111

sampler = EmbeddingComposite(DWaveSampler())
result = sampler.sample(prisma, num_reads=10, 
    anneal_schedule = reverse_schedule,
    reinitialize_state=False, initial_state = estado_inicial)

print("'Samples' obtenidos:\n")
print(result)

'Samples' obtenidos:

   0  1  2  3  4 energy num_oc. chain_.
0 +1 -1 +1 -1 -1   -4.0       1     0.0
2 +1 -1 +1 -1 -1   -4.0       1     0.0
3 +1 -1 +1 -1 -1   -4.0       1     0.0
4 -1 +1 -1 +1 +1   -4.0       1     0.0
5 -1 +1 -1 +1 -1   -4.0       1     0.0
6 -1 +1 -1 +1 -1   -4.0       1     0.0
7 -1 +1 -1 +1 +1   -4.0       1     0.0
8 -1 +1 -1 +1 +1   -4.0       1     0.0
9 -1 +1 -1 +1 +1   -4.0       1     0.0
1 +1 +1 +1 -1 -1   -2.0       1     0.0
['SPIN', 10 rows, 10 samples, 5 variables]


### Problema lineal binario con restricciones

In [5]:
sampler = EmbeddingComposite(DWaveSampler("Advantage_system4.1"))

# Definición del problema 

x0 = dimod.Binary("x0")
x1 = dimod.Binary("x1")
x2 = dimod.Binary("x2")

# Modelo CQM con restricciones
blp = dimod.ConstrainedQuadraticModel()

# Función objetivo
blp.set_objective(-5*x0+3*x1-2*x2)

# Restricciones
blp.add_constraint(x0 + x2 <= 1, "Primera restricción")
blp.add_constraint(3*x0 -x1 + 3*x2 <= 4, "Segunda restricción")

# Conversión a qubo (cqm-bqm) con multiplicador de Lagrange 
fl=10
qubo, invert = dimod.cqm_to_bqm(blp, lagrange_multiplier = fl)
result = sampler.sample(qubo, num_reads=100)

# Agregación de los n reads
samples = []
occurrences = []

for s in result.data():
    samples.append(invert(s.sample))
    occurrences.append(s.num_occurrences)
sampleset = dimod.SampleSet.from_samples_cqm(samples,blp,
    num_occurrences=occurrences)

In [6]:
# Resultados
print("\nFactor de Lagrange:", fl)
print("\nLas soluciones factibles al problema son:\n")
print(sampleset.filter(lambda s: s.is_feasible).aggregate())


Factor de Lagrange: 10

Las soluciones factibles al problema son:

  x0 x1 x2 energy num_oc. is_sat. is_fea.
0  1  0  0   -5.0      15 arra...    True
1  0  0  1   -2.0       9 arra...    True
2  1  1  0   -2.0      36 arra...    True
3  0  0  0    0.0       9 arra...    True
4  0  1  1    1.0      18 arra...    True
5  0  1  0    3.0      11 arra...    True
['INTEGER', 6 rows, 98 samples, 3 variables]



#### Influencia del factor de Lagrange:

Se va a repetir el problema usando factores de lagrange 4 y 1, que como sabemos, "amplifican" las penalizaciones por no cumplimiento de las restricciones.

In [133]:
# FL=4
fl=4
qubo, invert = dimod.cqm_to_bqm(blp, lagrange_multiplier = fl)
result = sampler.sample(qubo, num_reads=100)

# Agregación de los n reads
samples = []
occurrences = []

for s in result.data():
    samples.append(invert(s.sample))
    occurrences.append(s.num_occurrences)
sampleset = dimod.SampleSet.from_samples_cqm(samples,blp,
    num_occurrences=occurrences)


In [134]:
# Resultados
print("\nFactor de Lagrange:", fl)
print("\nLas soluciones factibles al problema son:\n")
print(sampleset.filter(lambda s: s.is_feasible).aggregate())


Las soluciones factibles al problema son:

  x0 x1 x2 energy num_oc. is_sat. is_fea.
0  1  0  0   -5.0      17 arra...    True
1  1  1  0   -2.0      27 arra...    True
2  0  0  1   -2.0      18 arra...    True
3  0  0  0    0.0      15 arra...    True
4  0  1  1    1.0      18 arra...    True
5  0  1  0    3.0       2 arra...    True
['INTEGER', 6 rows, 97 samples, 3 variables]


In [135]:
# FL1
fl=1
qubo, invert = dimod.cqm_to_bqm(blp, lagrange_multiplier = fl)
result = sampler.sample(qubo, num_reads=100)

# Agregación de los n reads
samples = []
occurrences = []

for s in result.data():
    samples.append(invert(s.sample))
    occurrences.append(s.num_occurrences)
sampleset = dimod.SampleSet.from_samples_cqm(samples,blp,
    num_occurrences=occurrences)

In [136]:
# Resultados
print("\nFactor de Lagrange:", fl)
print("\nLas soluciones factibles al problema son:\n")
print(sampleset.filter(lambda s: s.is_feasible).aggregate())


Factor de Lagrange: 1

Las soluciones factibles al problema son:

  x0 x1 x2 energy num_oc. is_sat. is_fea.
0  1  0  0   -5.0      71 arra...    True
1  0  0  1   -2.0       8 arra...    True
2  1  1  0   -2.0      17 arra...    True
3  0  0  0    0.0       1 arra...    True
4  0  1  1    1.0       1 arra...    True
['INTEGER', 5 rows, 98 samples, 3 variables]


## Anexo

### Samplers y solvers alternativos

Un `sampler` acepta un problema en formato de modelo cuadrático binario (BQM) o modelo cuadrático discreto (DQM) y devuelve asignaciones de variables. Los samplers generalmente intentan minimizar una función objetivo, pero también pueden muestrear distribuciones definidas por el problema.

https://docs.ocean.dwavesys.com/projects/system/en/stable/reference/samplers.html

https://docs.ocean.dwavesys.com/en/stable/docs_system/reference/samplers.html

#### SteepestDescentSolver()

https://docs.ocean.dwavesys.com/projects/greedy/en/latest/reference/generated/greedy.sampler.SteepestDescentSolver.sample.html

In [15]:
import greedy
import dimod
 
J = {(0,1):1, (0,3):1, (0,4):1,(1,2):1, (1,4):1, (2,3):1, (2,4):1, (3,4):1}
h = {}

prisma = dimod.BinaryQuadraticModel(h, J, 0.0, dimod.SPIN)

# Sampler con  SteepestDescentSolver

solver = greedy.SteepestDescentSolver()
solution = solver.sample(prisma, num_reads = 10)

print(solution.aggregate())

   0  1  2  3  4 energy num_oc. num_st.
0 +1 -1 +1 -1 +1   -4.0       3       1
1 -1 +1 -1 +1 +1   -4.0       1       2
2 -1 +1 -1 +1 -1   -4.0       3       1
3 +1 -1 +1 -1 -1   -4.0       3       1
['SPIN', 4 rows, 10 samples, 5 variables]


In [30]:
import tabu 

solver = tabu.TabuSampler()
solution = solver.sample(prisma, num_reads = 15)

print(solution.aggregate())

   0  1  2  3  4 energy num_oc. num_re.
0 +1 -1 +1 -1 -1   -4.0       5       1
1 -1 +1 -1 +1 +1   -4.0       4       1
2 +1 -1 +1 -1 +1   -4.0       4       1
3 -1 +1 -1 +1 -1   -4.0       2       1
['SPIN', 4 rows, 15 samples, 5 variables]


#### SimulatedAnnealingSampler()

Sampler dimod que utiliza un algoritmo de annealing simulado, un método heurístico de optimización para ordenadores clásicos.

https://docs.ocean.dwavesys.com/projects/neal/en/latest/reference/sampler.html

In [32]:
import neal 

sampler = neal.SimulatedAnnealingSampler()
solution = sampler.sample(prisma, num_reads = 10)

print(solution.aggregate())

   0  1  2  3  4 energy num_oc.
0 +1 -1 +1 -1 -1   -4.0       3
1 -1 +1 -1 +1 +1   -4.0       4
2 +1 -1 +1 -1 +1   -4.0       1
3 -1 +1 -1 +1 -1   -4.0       2
['SPIN', 4 rows, 10 samples, 5 variables]


ValueError: invalid combination of arguments provided: if data capture not enabled, problem/response/solver have to be specified; also, make sure a structured problem is being inspected

#### Samplers Dwave

https://docs.ocean.dwavesys.com/en/stable/docs_system/reference/samplers.html

In [25]:
import dwave.system 

sampler = dwave.system.LeapHybridSampler()


In [22]:
import dwave.system 
'''
sampler = DWaveSampler()

Problem graph incompatible with solver. 
Please use 'EmbeddingComposite' to map the problem graph to the solver
'''
sampler = EmbeddingComposite(DWaveSampler())
solution = sampler.sample(prisma, num_reads = 10)

print(solution.aggregate())

   0  1  2  3  4 energy num_oc. chain_.
0 -1 +1 -1 +1 +1   -4.0       2     0.0
1 -1 +1 -1 +1 -1   -4.0       7     0.0
2 +1 -1 +1 -1 +1   -4.0       1     0.0
['SPIN', 3 rows, 10 samples, 5 variables]


In [141]:
sampler.properties["quota_conversion_rate"]


20

In [ ]:

D-Wave system como sampler para modelos BQM 

#### DWaveCliqueSampler

Sampler para resolver Clique BQM en los sistemas D-Wave

